In [60]:
from selenium import webdriver
from lxml import etree
import requests
import time
import re
import pandas as pd

In [ ]:
driver = webdriver.Chrome()   # 启动chrome浏览器
url = 'https://movie.douban.com/subject/26266893/comments?status=P'
driver.get(url)   # 获取网页源码数据
dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))   

In [55]:

names = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/text()')  # 用户名
# span[2]表示第二个span标签，/@class表示取出class属性中的内容
ratings = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[2]/@class')   # 评分
times = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[@class="comment-time "]/@title')     # 评论发布时间
message = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="short"]/text()')    # 短评正文
user_url = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/@href')           # 用户主页网址
votes = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="votes"]/text()')    # 赞同数量

cookies_str = 'bid=wO-iCKezZeE; ap_v=0,6.0; __utmc=30149280; __utmc=223695111; __utmz=223695111.1560843473.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _pk_ses.100001.4cf6=*; __utma=30149280.868676413.1560843473.1560843473.1560846159.2; __utmz=30149280.1560846159.2.2.utmcsr=movie.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/subject/26266893/comments; __utmt=1; __utma=223695111.378965285.1560843473.1560843473.1560846213.2; __utmb=223695111.0.10.1560846213; __utmb=30149280.3.10.1560846159; dbcl2="171190284:Iq7vZ7fczlA"; ck=FUUE; _pk_id.100001.4cf6=541bc214cc05489b.1560843473.2.1560846535.1560843473.; push_noty_num=0; push_doumail_num=0'
cookies = {}
for i in cookies_str.split(';'):  # 将cookies整理成所需格式，这里整理为字典格式
    k, v = i.split('=', 1)  # 1表示只按照第一个等号进行拆分，后面如果还有等号则不进行拆分
    cookies[k] = v


city = []  # 用户居住地列表
load_times = []  #  用户注册时间列表
for i in user_url:
    web_data = requests.get(i, cookies=cookies, headers=header)  
    dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))   
    address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')                    # 用户居住地
    load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')   # 用户注册时间
    city.append(address)
    load_times.append(load_time)
    time.sleep(0.1)  #   设置休息时间，避免被反爬虫

In [56]:
# 来获取具体评分中的数字，由于有的用户没用评分所以返回空,findall返回的是一个列表
ratings = ["" if "rating" not in i else re.findall('[0-9]{2}', i)[0] for i in ratings]
# 来获取登录时间的具体时间数据
load_times = ["" if i == [] else i[1].strip()[:-2] for i in load_times]  # strip()方法可以去除字符串两边的空格
load_times

['2005-07-18',
 '2008-08-30',
 '2018-10-07',
 '2006-08-12',
 '2008-03-12',
 '2008-03-04',
 '2012-08-07',
 '2009-04-14',
 '2012-11-21',
 '2012-07-22',
 '',
 '2010-12-26',
 '2009-12-20',
 '2007-08-10',
 '2006-07-22',
 '2005-10-09',
 '2009-12-01',
 '2009-09-11',
 '2007-12-07',
 '2009-01-25']

In [58]:
city = ["" if i == [] else i[0] for i in city]

['北京',
 '北京',
 '北京',
 '北京',
 '北京',
 '北京',
 '重庆',
 '北京',
 '上海',
 '陕西西安',
 '',
 '',
 '广东深圳',
 '北京',
 '上海',
 "P'yŏngyang, North Korea",
 '上海',
 '北京',
 '',
 '']

In [62]:
data = pd.DataFrame({
        '用户名': names,
        '居住城市': city,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
data

,用户名,居住城市,加入时间,评分,发表时间,短评正文,赞同数量
0,影志,北京,2005-07-18,40,2019-02-04 15:56:16,“北京道路安全委提醒你：道路千万条，安全第一条，行车不规范，亲人两行泪” 这句广播语真是又土...,71752
1,陆支羽,北京,2008-08-30,50,2019-01-29 20:10:48,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,62246
2,沙雕电影,北京,2018-10-07,40,2019-02-05 00:24:35,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,41028
3,frozenmoon,北京,2006-08-12,30,2019-02-05 16:29:48,1.从特效和技术上讲，这应该是迄今为止此类中国电影的巅峰了。磅礴恢宏，细节营造用心。2.故事...,4396
4,乌鸦火堂,北京,2008-03-12,50,2019-01-20 19:00:54,华语真正意义上的第一部科幻大片！刘慈欣的硬核科幻设定，小兵扛大旗的主流价值观人设，东方式的家...,15026
5,张小北,北京,2008-03-04,50,2019-01-29 02:11:13,从各个方面来说都是一部好看的类型片。而最特别的是它是一部国产科幻电影，影迷+科幻迷的双重满足...,12909
6,凌睿,重庆,2012-08-07,40,2019-01-27 14:16:11,《流浪地球》小说本就想象力丰富，让地球连同地球人一同逃难的创意荡气回肠、惊艳无比。\n电影中...,12874
7,亵渎电影,北京,2009-04-14,40,2019-02-02 14:29:09,中国导演能拍出这样的硬科幻，看到这样的完成度，想想就激动，可以让人原谅一切不完美！说这部电影...,9038
8,刘小流,上海,2012-11-21,40,2019-01-20 14:56:12,宏大又动人。为原著赋予了细腻的想象，整体气质比美式末日片都多了一份安静和真实，也有萦绕不去的...,5155
9,江寒园,陕西西安,2012-07-22,30,2019-02-05 14:31:49,1. 煽情、尴尬与生硬并存\n2.主要是2001太空漫游（致敬AI杀人太空戏）和罗兰.艾默里...,3927
